In [1]:
# Transcript
from youtube_transcript_api import YouTubeTranscriptApi
import os
# Summarization
import pandas as pd
import numpy as np
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
from langchain import PromptTemplate,  LLMChain

In [2]:
def save_transcript_to_file(transcript_text, file_name):
    temp_folder = 'temp'
    file_path = os.path.join(temp_folder, file_name)
    # Create the 'temp' directory if it doesn't exist
    if not os.path.exists(temp_folder):
        os.makedirs(temp_folder)
    try:
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(transcript_text)
        return file_path
    except Exception as e:
        return str(e)

def fetch_transcript(video_url):
    try:
        # Extract the video ID from the URL
        video_id = video_url.split("v=")[1]
        # Fetch the transcript for the video
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        # Process the transcript data
        text_transcript = "\n".join([entry['text'] for entry in transcript])
        return text_transcript
    except Exception as e:
        return str(e)

In [4]:
# Define the YouTube video URL (replace with your desired video URL)
# video_url = "https://www.youtube.com/watch?v=13CZPWmke6A&list=WL&index=15&t=8s&ab_channel=LexFridman"
video_url = "https://www.youtube.com/watch?v=ZsMJBw4faJM" # How To Self Study FAST

transcript_text = fetch_transcript(video_url)
if transcript_text:
    print("Transcript:\n")
    print(transcript_text[0:100])
    save_transcript_to_file(transcript_text, "video_transcript.txt")
else:
    print("Failed to fetch the transcript.")

Transcript:

you can use Neuroscience to hack your
brain to self-study way faster and you
can use AI to enhance t


In [5]:
model = "meta-llama/Llama-2-7b-chat-hf"
#
tokenizer = AutoTokenizer.from_pretrained(model)
max_token_length = 1000

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=max_token_length,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})
template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
           """
prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [13]:
transcript_text = transcript_text.replace("\n"," ")

In [6]:
summary = llm_chain.run(transcript_text[0:1000])
summary

In [9]:
transcript_text[0:500]

"you can use Neuroscience to hack your\nbrain to self-study way faster and you\ncan use AI to enhance those effects even\nmore as a child I absolutely hated\nschool have you ever experienced that\nfeeling when you're not paying attention\nand that a teacher calls on you for a\nquestion and you just like don't know\nwhat the teacher asked H so your choices\nare either say something random or ask a\nteacher to repeat themselves neither of\nwhich are good choices to do but I\nalways did well in school though I\n"

In [7]:
summary

' • Using neuroscience to hack your brain to study faster.\n            • Using AI to even enhance those effects.\n            • The author hated school as a child.\n            • Struggling to pay attention and recall information.\n            • Feeling anxious when called on in class.\n            • The author always did well in school despite these challenges.\n\nPlease provide the actual text you would like me to summarize and I will assist you.'

In [18]:
transcript_text_list = transcript_text.split('\n')
pd.DataFrame(data=transcript_text_list,columns=['text'])

,text
0,the following is a conversation with
1,elias discover
2,co-founder and chief scientist of open
3,ai one of the most cited computer
4,scientists in history
...,...
2960,if this were then subjected to an
2961,appropriate course of education
2962,one would obtain the adult brain
2963,thank you for listening and hope to see
